In [2]:
import mitsuba as mi
import drjit as dr 
import matplotlib.pyplot as plt
mi.set_variant("cuda_ad_rgb")

# Camera Parameters Setup

In [3]:
Intrinsics = mi.Matrix3f([[422.3949, 0,313.5413],[0,422.4885,233.1770],[0,0,1]])
Z_camera = -44
image_res = (640,480)
cam_conner1 = dr.inverse(Intrinsics)@mi.Point3f(0,0,1)*-Z_camera
cam_conner2 = dr.inverse(Intrinsics)@mi.Point3f(image_res[0]-1,image_res[1]-1,1)*-Z_camera


# ### Image pixel resolution ###
# image_res = (256,256)
# ### camera origin          ###
cam_origin = mi.Point3f(0,0,Z_camera)
cam_range = ((cam_conner1[0,0],cam_conner1[1,0]),(cam_conner2[0,0],cam_conner2[1,0]))
cam_fov= (cam_conner2[0,0]-cam_conner1[0,0],cam_conner2[1,0]-cam_conner1[1,0])
from utils import camera 
cam = camera.Camera(cam_origin, image_res,cam_range)

# LED Position Setup

In [4]:
import utils.led_ring

N_LED = 24
LED0_pos = mi.Point3f(34.41,0,-12.5)
LED_ring = utils.led_ring.LED_params(N_LED, LED0_pos)

# Sensor Setup

In [5]:
from utils import iristac
sensor = iristac.sensor(cam,LED_ring)

# Reference Image Generation

In [ ]:
i_LED=10


LED_ring.E[:,:,:,:]=LED_ring.E.array*0
LED_ring.E[:,i_LED,:,2]=4
sensor.set_LED(LED_ring)
# I_ref = sensor.get_tactile_img()
# plt.imshow(I_ref+rng.next_float32()/5)
# plt.show()
noise_index = dr.arange(mi.UInt,dr.prod(image_res)*3)

for i_LED in range(24):
    LED_ring.E[:,:,:,:]=LED_ring.E.array*0
    LED_ring.E[:,i_LED,:,:]=LED_ring.E[:,i_LED,:,:].array+4
    sensor.set_LED(LED_ring)
    I_0 = sensor.get_tactile_img()
    for i_intensity in range(256):
        I_noise = I_0*i_intensity/256
        I_noise[:,:,0] = I_noise[:,:,0].array*1.15
        I_noise[:,:,1] = I_noise[:,:,1].array*1.05
        I_noise[:,:,2] = I_noise[:,:,2].array*0.9
        noise = mi.sample_tea_float32(i_LED*256+i_intensity,noise_index)-0.5
        I_noise = I_noise+noise/10
        for i_RGB in range(3):
            I_save = mi.TensorXf(0,I_0.shape)
            I_save[:,:,i_RGB] = I_noise[:,:,i_RGB].array
            path = "../data/calibration_sim/{0:02d}_{1:03d}_{2:01d}.jpg".format(i_LED,i_intensity,i_RGB)
            bmp = mi.Bitmap(I_save)
            bmp = bmp.convert(mi.Bitmap.PixelFormat.RGB, mi.Struct.Type.UInt8, False)
            bmp.write(path)
            print("\r", end=path, flush=True)